_한국어로 기계번역됨_


# ReAct 에이전트를 처음부터 만드는 방법 (기능적 API)

!!! 정보 "전제 조건"
    이 가이드는 다음에 대한 이해를 전제로 합니다:
    
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models)
    - [메시지](https://python.langchain.com/docs/concepts/messages)
    - [도구 호출](https://python.langchain.com/docs/concepts/tool_calling/)
    - [진입점](../../concepts/functional_api/#entrypoint) 및 [작업](../../concepts/functional_api/#task)

이 가이드는 LangGraph [기능적 API](../../concepts/functional_api)를 사용하여 ReAct 에이전트를 구현하는 방법을 보여줍니다.

ReAct 에이전트는 다음과 같이 작동하는 [도구 호출 에이전트](../../concepts/agentic_concepts/#tool-calling-agent)입니다:

1. 쿼리가 채팅 모델에 발행됩니다;
2. 모델이 [도구 호출](../../concepts/agentic_concepts/#tool-calling)을 생성하지 않으면, 모델의 응답을 반환합니다.
3. 모델이 도구 호출을 생성하면, 사용 가능한 도구로 도구 호출을 실행하고, 이를 메시지 목록에 [도구 메시지](https://python.langchain.com/docs/concepts/messages/)로 추가한 후, 이 과정을 반복합니다.

이것은 메모리, 인간 참여 루프 기능 및 기타 기능으로 확장할 수 있는 간단하고 다재다능한 설정입니다. 예제는 전용 [방법 안내서](../../how-tos/#prebuilt-react-agent)를 참조하세요.

## 설정

먼저, 필요한 패키지를 설치하고 API 키를 설정합시다:


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
     <p class="admonition-title">더 나은 디버깅을 위해 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
     <p style="padding-top: 5px;">
         LangSmith에 가입하여 문제를 신속하게 파악하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith는 LangGraph로 구축된 LLM 앱을 디버깅하고 테스트하며 모니터링하는 데 사용할 수 있는 추적 데이터를 제공합니다. 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">문서</a>를 참조하세요.
     </p>
 </div>


## ReAct 에이전트 생성

필요한 패키지를 설치하고 환경 변수를 설정했으니, 이제 에이전트를 생성할 수 있습니다.

### 모델 및 도구 정의

먼저 예제에 사용할 도구와 모델을 정의해 보겠습니다. 여기에서는 위치의 날씨에 대한 설명을 가져오는 단일 플레이스홀더 도구를 사용할 것입니다.

이번 예제에서는 [OpenAI](https://python.langchain.com/docs/integrations/providers/openai/) 채팅 모델을 사용할 것이지만, 도구 호출을 [지원하는](https://python.langchain.com/docs/integrations/chat/) 모든 모델도 괜찮습니다.


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")


@tool
def get_weather(location: str):
    """Call to get the weather from a specific location."""
    # This is a placeholder for the actual implementation
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny!"
    elif "boston" in location.lower():
        return "It's rainy!"
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]


### 작업 정의

다음으로 우리가 실행할 [작업](../../concepts/functional_api/#task)을 정의합니다. 여기에는 두 가지 다른 작업이 있습니다:

1. **모델 호출**: 우리는 메시지 목록을 사용하여 채팅 모델에 쿼리를 하고자 합니다.
2. **도구 호출**: 모델이 도구 호출을 생성하는 경우, 우리는 그것을 실행하고자 합니다.


In [2]:
from langchain_core.messages import ToolMessage
from langgraph.func import entrypoint, task

tools_by_name = {tool.name: tool for tool in tools}


@task
def call_model(messages):
    """Call model with a sequence of messages."""
    response = model.bind_tools(tools).invoke(messages)
    return response


@task
def call_tool(tool_call):
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call["args"])
    return ToolMessage(content=observation, tool_call_id=tool_call["id"])


### 진입점 정의

우리의 [진입점](../../concepts/functional_api/#entrypoint)은 이 두 작업의 조정을 처리할 것입니다. 위에서 설명한 바와 같이, `call_model` 작업이 도구 호출을 생성할 때, `call_tool` 작업은 각 도구에 대한 응답을 생성합니다. 우리는 모든 메시지를 단일 메시지 리스트에 추가합니다.

!!! 팁
    작업이 미래와 유사한 객체를 반환하기 때문에, 아래 구현은 도구를 병렬로 실행합니다.


In [3]:
from langgraph.graph.message import add_messages


@entrypoint()
def agent(messages):
    llm_response = call_model(messages).result()
    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_result_futures = [
            call_tool(tool_call) for tool_call in llm_response.tool_calls
        ]
        tool_results = [fut.result() for fut in tool_result_futures]

        # Append to message list
        messages = add_messages(messages, [llm_response, *tool_results])

        # Call model again
        llm_response = call_model(messages).result()

    return llm_response


## 사용법

우리 에이전트를 사용하려면 메시지 목록으로 호출합니다. 우리의 구현에 따라, 이것들은 LangChain [메시지](https://python.langchain.com/docs/concepts/messages/) 객체이거나 OpenAI 스타일의 딕셔너리일 수 있습니다:


In [4]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message]):
    for task_name, message in step.items():
        if task_name == "agent":
            continue  # Just print task updates
        print(f"\n{task_name}:")
        message.pretty_print()


{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_tNnkrjnoz6MNfCHJpwfuEQ0v)
 Call ID: call_tNnkrjnoz6MNfCHJpwfuEQ0v
  Args:
    location: san francisco

call_tool:
================================= Tool Message =================================

It's sunny!

call_model:
================================== Ai Message ==================================

The weather in San Francisco is sunny!


완벽합니다! 그래프가 `get_weather` 도구를 올바르게 호출하고 도구로부터 정보를 받은 후 사용자에게 응답합니다. LangSmith 추적을 [여기서](https://smith.langchain.com/public/d5a0d5ea-bdaa-4032-911e-7db177c8141b/r) 확인해 보세요.


## 스레드 수준 지속성 추가

스레드 수준 지속성을 추가하면 에이전트와의 대화 경험을 지원할 수 있습니다. 이후의 호출은 이전 메시지 목록에 추가되어 전체 대화 맥락을 유지합니다.

에이전트에 스레드 수준 지속성을 추가하려면:

1. 체크포인터를 선택합니다: 여기서는 간단한 인메모리 체크포인터인 [MemorySaver](../../reference/checkpoints/#langgraph.checkpoint.memory.MemorySaver)를 사용할 것입니다.
2. 이전 메시지 상태를 두 번째 인수로 받을 수 있도록 진입점을 업데이트합니다. 여기서는 메시지 업데이트를 이전 메시지 시퀀스에 단순히 추가합니다.
3. 워크플로우에서 어떤 값을 반환할지와 체크포인터에 의해 `previous`로 저장될 값을 선택합니다. (선택 사항)


In [5]:
from langgraph.checkpoint.memory import MemorySaver

# highlight-next-line
checkpointer = MemorySaver()


# highlight-next-line
@entrypoint(checkpointer=checkpointer)
# highlight-next-line
def agent(messages, previous):
    # highlight-next-line
    if previous is not None:
        # highlight-next-line
        messages = add_messages(previous, messages)

    llm_response = call_model(messages).result()
    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_result_futures = [
            call_tool(tool_call) for tool_call in llm_response.tool_calls
        ]
        tool_results = [fut.result() for fut in tool_result_futures]

        # Append to message list
        messages = add_messages(messages, [llm_response, *tool_results])

        # Call model again
        llm_response = call_model(messages).result()

    # Generate final response
    messages = add_messages(messages, llm_response)
    # highlight-next-line
    return entrypoint.final(value=llm_response, save=messages)


이제 애플리케이션을 실행할 때 구성(config)을 전달해야 합니다. 이 구성은 대화 스레드를 식별하는 식별자를 지정합니다.

!!! 팁

    스레드 수준 지속성에 대한 자세한 내용은 우리의 [개념 페이지](../../concepts/persistence/)와 [사용 방법 가이드](../../how-tos/#persistence)를 참조하세요.


In [6]:
config = {"configurable": {"thread_id": "1"}}


우리는 이전과 동일한 방식으로 쓰레드를 시작하지만, 이번에는 설정을 전달합니다:


In [7]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

# highlight-next-line
for step in agent.stream([user_message], config):
    for task_name, message in step.items():
        if task_name == "agent":
            continue  # Just print task updates
        print(f"\n{task_name}:")
        message.pretty_print()


{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_lubbUSdDofmOhFunPEZLBz3g)
 Call ID: call_lubbUSdDofmOhFunPEZLBz3g
  Args:
    location: San Francisco

call_tool:
================================= Tool Message =================================

It's sunny!

call_model:
================================== Ai Message ==================================

The weather in San Francisco is sunny!


후속 대화를 요청할 때, 모델은 이전 맥락을 사용하여 우리가 날씨에 대해 묻고 있다는 것을 추론합니다.


In [8]:
user_message = {"role": "user", "content": "How does it compare to Boston, MA?"}
print(user_message)

for step in agent.stream([user_message], config):
    for task_name, message in step.items():
        if task_name == "agent":
            continue  # Just print task updates
        print(f"\n{task_name}:")
        message.pretty_print()


{'role': 'user', 'content': 'How does it compare to Boston, MA?'}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_8sTKYAhSIHOdjLD5d6gaswuV)
 Call ID: call_8sTKYAhSIHOdjLD5d6gaswuV
  Args:
    location: Boston, MA

call_tool:
================================= Tool Message =================================

It's rainy!

call_model:
================================== Ai Message ==================================

Compared to San Francisco, which is sunny, Boston, MA is experiencing rainy weather.


[LangSmith 추적](https://smith.langchain.com/public/20a1116b-bb3b-44c1-8765-7a28663439d9/r)에서 각 모델 호출에서 전체 대화 맥락이 유지되는 것을 볼 수 있습니다.
